In [ ]:
#Requests: This library allows you to send HTTP requests using Python. It's used in the script to fetch web pages.
!pip install requests


In [ ]:
#BeautifulSoup: This library is used for web scraping, allowing you to parse HTML and XML documents. It's used to extract data from the fetched web pages.
#re (Regular Expressions): This module is a built-in Python library, so you don't need to install it. It's used for string searching and manipulation.
!pip install beautifulsoup4


In [ ]:
#Schedule: This library allows you to schedule tasks. In this script, it's used to automate the scraping process at a scheduled time.
!pip install schedule


In [ ]:
#time: This is another built-in Python module used for time-related tasks. No installation is required.

#datetime: This built-in module is used to work with dates and times.

In [ ]:
#Tabulate: This library is used to print tabulated data in a structured format, which is particularly useful for displaying the scraping results in a readable way.
!pip install tabulate


In [ ]:
#Required libraries
!pip install requests beautifulsoup4 schedule tabulate


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from tabulate import tabulate

def scrape_kohls_product(url, product_name):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        price_element = soup.find("span", class_="pdpprice-row2-main-text")
        price = price_element.text.strip() if price_element else "N/A"
        rating_pattern = r'"avgRating":"(\d+\.\d+)"'
        rating_match = re.search(rating_pattern, str(soup))
        rating = rating_match.group(1) if rating_match else "N/A"

        return [product_name, price, rating]
    else:
        print("Error: Could not access the webpage.")
        return [product_name, "N/A", "N/A"]

def scrape_lovelyskin_product(url, product_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            price_match = re.search(r'"Price":"(.*?)",', response.text)
            rating_match = re.search(r'"RatingValue":"(.*?)",', response.text)
            price = price_match.group(1) if price_match else "N/A"
            rating = rating_match.group(1) if rating_match else "N/A"
            return [product_name, f"USD {price}", rating]
    except Exception as e:
        print(f"Error: {e}")
        return [product_name, "Error", "Error"]

def scrape_skinlovecream_product(url, product_name):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        price_tag = soup.find("meta", property="og:price:amount")
        currency_tag = soup.find("meta", property="og:price:currency")
        rating_div = soup.find("div", class_="jdgm-rev-widg")

        price = price_tag["content"] if price_tag else "N/A"
        currency = currency_tag["content"] if currency_tag else "N/A"
        rating = rating_div["data-average-rating"] if rating_div else "N/A"

        return [product_name, f"{currency} {price}", rating]
    else:
        return [product_name, "N/A", "N/A"]

def scrape_walgreens_product(url, product_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            html_content = str(soup)

            price_pattern = r'\$\d+\.\d{2}'
            price = re.findall(price_pattern, html_content)
            found_price = price[0] if price else "Price not found"

            rating_pattern = r'"shortMessage":"([\d.]+ out of 5)"'
            rating_match = re.search(rating_pattern, html_content)
            found_rating = rating_match.group(1).replace(" out of 5", "") if rating_match else "Rating not found"

            return [product_name, found_price, found_rating]
        else:
            return [product_name, "Error", "Error"]
    except Exception as e:
        return [product_name, "Error", "Error"]

def scrape_ultabeauty_product(url, product_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text

            price_pattern = r'<span class="Text-ds Text-ds--title-6 Text-ds--left Text-ds--black">\$(\d+\.\d{2}) </span>'
            rating_pattern = r'"aggregateRating":{"@type":"AggregateRating","ratingValue":(\d+\.\d)'

            price_match = re.search(price_pattern, html_content)
            rating_match = re.search(rating_pattern, html_content)

            found_price = f'${price_match.group(1)}' if price_match else "Price not found"
            found_rating = rating_match.group(1) if rating_match else "Rating not found"

            return [product_name, found_price, found_rating]
        else:
            return [product_name, "Error", "Error"]
    except Exception as e:
        return [product_name, "Error", "Error"]

# Combined scraping function
def combined_scrape():
    print(f" Scraping started for Kohls, Lovelyskin, Skinlovecream, Walgreens, Ultabeauty at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Kohls products
    kohls_products = [
        ("https://www.kohls.com/product/prd-5050008/olaplex-no-7-bonding-hair-oil.jsp?skuid=17911732&CID=shopping15&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=17911732&utm_campaignid=20485373179&CID=shopping30&utm_campaign=MediumSSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20485373179&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTchpQ_E0QOv8B4c1oPWYYi4Fr-60QRdAuCuDpeJorwSAXAQ-4posn0aAn9sEALw_wcB&gclsrc=aw.ds", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
       ("https://www.kohls.com/product/prd-5050005/olaplex-no-4-bond-maintenance-shampoo.jsp?skuid=67270281&CID=shopping30&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=67270281&utm_campaignid=20507263090&CID=shopping30&utm_campaign=SSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20507263090&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTdfObfjg7NJgEgbftJ7zSZEO5Kbm42pdU7mpo-oP61ddx_saVyqJhIaAu0zEALw_wcB&gclsrc=aw.ds", "Olaplex No. 4 Bond Maintenance Shampoo 8.5 fl oz"),
       ("https://www.kohls.com/product/prd-5050006/olaplex-no-5-bond-maintenance-conditioner.jsp?skuid=65150746&CID=shopping15&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=65150746&utm_campaignid=20485373179&CID=shopping30&utm_campaign=MediumSSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20485373179&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTfHKiLgn1tIc_JhVFD_SSPI-Xs3KzDqdR6VxoCl5Q-Yyc9RkmDE6cMaAv-7EALw_wcB&gclsrc=aw.ds", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
       ("https://www.kohls.com/product/prd-5532929/olaplex-no-6-bond-smoother-reparative-styling-creme.jsp?skuid=71250025&CID=shopping30&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=71250025&utm_campaignid=20507263090&CID=shopping30&utm_campaign=SSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20507263090&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTf94yPAHlFCvJ--MUTq11aFNtqulBRmfO8iTpUh6lkngel6vXBjo1YaAkedEALw_wcB&gclsrc=aw.ds", "Olaplex No. 6 Bond Smoother 3.3 fl oz"),
       ("https://www.kohls.com/product/prd-5050004/olaplex-no-3-hair-repair-perfector.jsp?skuid=49701743&CID=shopping30&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=49701743&utm_campaignid=20507263090&CID=shopping30&utm_campaign=SSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20507263090&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTcR-zMBFZhx7NXZdAtSKIU37Xgyp8pH06g4AU2bwteC4KqcbF-j7ToaAg_dEALw_wcB&gclsrc=aw.ds", "No. 3 Hair Perfector Treatment 3.3 fl oz")

    ]
    kohls_data = [scrape_kohls_product(url, name) for url, name in kohls_products]
    print(tabulate(kohls_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    # Lovelyskin products
    lovelyskin_products = [
        ("https://www.lovelyskin.com/o/olaplex-no-7-bonding-oil", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
        ("https://www.lovelyskin.com/o/olaplex-no-4-bond-maintenance-shampoo?ref=gbase", "Olaplex No. 4 Bond Maintenance Shampoo Size: 8.5 fl oz"),
        ("https://www.lovelyskin.com/o/olaplex-no-5-bond-maintenance-conditioner", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
        ("https://www.lovelyskin.com/o/olaplex-no-6-bond-smoother?ref=gbase&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTeAH7knJVVWPFjytwdkhdksfBX_0uNRx8OhQJyiK-wylUZE8x6TRlAaAmIMEALw_wcB", "Olaplex No. 6 Bond Smoother Size: 3.3 fl oz"),
        ("https://www.lovelyskin.com/o/olaplex-no-3-hair-perfector", "No. 3 Hair Perfector Treatment Size: 3.3 fl oz")

    ]
    lovelyskin_data = [scrape_lovelyskin_product(url, name) for url, name in lovelyskin_products]
    print(tabulate(lovelyskin_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    # Skinlovecream products
    skinlovecream_products = [
        ("https://skinlovecream.com/products/olaplex-no-7-bonding-oil?variant=42577740792054&currency=USD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTflJA-jxmndNNH2Vb43Shu-4vB_OE95yZOrzWTaHrioodCLZA3iiOcaAmuQEALw_wcB", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
        ("https://skinlovecream.com/products/olaplex-no-4-bond-maintenance-shampoo?variant=42577567842550&currency=USD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTddLO8wcbqkX1B87HEzC3R9w1cLS6ZcrV_ZmemfjgxqMz3XM3QS1pkaAuM8EALw_wcB", "Olaplex No. 4 Bond Maintenance Shampoo Size: 8.5 fl oz"),
        ("https://skinlovecream.com/products/olaplex-no-5-bond-maintenance-conditioner?variant=42577632395510", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
        ("https://skinlovecream.com/products/olaplex-no-6-bond-smoother?variant=42577691246838", "Olaplex No. 6 Bond Smoother Size: 3.3 fl oz"),
        ("https://skinlovecream.com/products/olaplex-no-3-hair-perfector?variant=42577661886710", "No. 3 Hair Perfector Treatment Size: 3.3 fl oz")

    ]
    skinlovecream_data = [scrape_skinlovecream_product(url, name) for url, name in skinlovecream_products]
    print(tabulate(skinlovecream_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    # Walgreens products
    walgreens_products = [
        ("https://www.walgreens.com/store/c/olaplex-no.-7-bonding-hair-oil/ID=300425433-product", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
        ("https://www.walgreens.com/store/c/olaplex-no.-4-bond-maintenance-shampoo/ID=300427190-product?ext=gooFY23_GOO_RET_RETAILDEMANDGEN_Performance%2BMax%2B-%2BBeauty_REV_SRC_PMAX_PMAX_NA_PMAX_ENG__pla_online&gclsrc=aw.ds&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTdLmST044rktqFajOY_dfaV89unsRjC6SaNBqgB-vJcmQ2dkMOCm8saAj0PEALw_wcB", "Olaplex No. 4 Bond Maintenance Shampoo Size: 8.5 fl oz"),
        ("https://www.walgreens.com/store/c/olaplex-no.-5-bond-maintenance-conditioner/ID=300429026-product", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
        ("https://www.walgreens.com/store/c/olaplex-no.-6-bond-smoother/ID=300427188-product?ext=gooFY%7E24_LB%7ERDG_CH%7ESEARCH_CN%7ERDG_Catch%2BAll%2B-%2BRSC_CA%7EFOS_MT%7EPLA_LG%7EEN1_RE%7ENA_MK%7EGM_OB%7ESALES_PK%7EOMNIREV_KT%7ENA_KM%7ENA_AS%7EGOOFY%7E24_%2BAS%7EGOO_FF%7ECatch%2BAll%2B-%2BAll%2BProducts_DT%7ENA_LG%7EEN1_MK%7EGM_SA%7ETP_SI%7ENA_TC%7EKEY_PB%7EGO_PK%7EOMNIREV__pla_online&gclsrc=aw.ds&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTf1aX6NoZO1MRQti8xYoLNfTJyn1TvXY7yyxLG5UzGvEWPOMbuSvN0aAhSzEALw_wcB", "Olaplex No. 6 Bond Smoother Size: 3.3 fl oz"),
        ("https://www.walgreens.com/store/c/olaplex-no.-3-hair-perfector-treatment/ID=prod6310485-product", "No. 3 Hair Perfector Treatment Size: 3.3 fl oz")

    ]
    walgreens_data = [scrape_walgreens_product(url, name) for url, name in walgreens_products]
    print(tabulate(walgreens_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    # Ultabeauty products
    ultabeauty_products = [
        ("https://www.ulta.com/p/no7-bonding-oil-pimprod2029354?sku=2591018", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
        ("https://www.ulta.com/p/no4-bond-maintenance-shampoo-pimprod2029348?sku=2591015", "Olaplex No. 4 Bond Maintenance Shampoo Size: 8.5 fl oz"),
        ("https://www.ulta.com/p/no5-bond-maintenance-conditioner-pimprod2029349?sku=2591016", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
        ("https://www.ulta.com/p/no6-bond-smoother-pimprod2030408?sku=2592990", "Olaplex No. 6 Bond Smoother Size: 3.3 fl oz"),
        ("https://www.ulta.com/p/no3-hair-perfector-pimprod2029343?sku=2591014", "No. 3 Hair Perfector Treatment Size: 3.3 fl oz")

    ]
    ultabeauty_data = [scrape_ultabeauty_product(url, name) for url, name in ultabeauty_products]
    print(tabulate(ultabeauty_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    print(f" Scraping ended at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# Execute the combined scraping function
combined_scrape()


 Scraping started for Kohls, Lovelyskin, Skinlovecream, Walgreens, Ultabeauty at 2024-03-29 01:43:58

╒════════════════════════════════════════════════════════════╤═════════╤══════════╕
│ Product Name                                               │ Price   │   Rating │
╞════════════════════════════════════════════════════════════╪═════════╪══════════╡
│ Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz                │ $30.00  │      4.3 │
├────────────────────────────────────────────────────────────┼─────────┼──────────┤
│ Olaplex No. 4 Bond Maintenance Shampoo 8.5 fl oz           │ $30.00  │      4.2 │
├────────────────────────────────────────────────────────────┼─────────┼──────────┤
│ Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz │ $30.00  │      4.1 │
├────────────────────────────────────────────────────────────┼─────────┼──────────┤
│ Olaplex No. 6 Bond Smoother 3.3 fl oz                      │ $30.00  │      4.4 │
├─────────────────────────────────────────────────────────

In [16]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Print the current date and time
print("Current date and time:", now)


Current date and time: 2024-03-29 02:05:11.038981


In [15]:
#Scrape with schedule
#Run the code for scraping
import requests
from bs4 import BeautifulSoup
import re
import schedule
import time
from datetime import datetime
from tabulate import tabulate

def scrape_kohls_product(url, product_name):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        price_element = soup.find("span", class_="pdpprice-row2-main-text")
        price = price_element.text.strip() if price_element else "N/A"
        rating_pattern = r'"avgRating":"(\d+\.\d+)"'
        rating_match = re.search(rating_pattern, str(soup))
        rating = rating_match.group(1) if rating_match else "N/A"

        return [product_name, price, rating]
    else:
        print("Error: Could not access the webpage.")
        return [product_name, "N/A", "N/A"]

def scrape_lovelyskin_product(url, product_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            price_match = re.search(r'"Price":"(.*?)",', response.text)
            rating_match = re.search(r'"RatingValue":"(.*?)",', response.text)
            price = price_match.group(1) if price_match else "N/A"
            rating = rating_match.group(1) if rating_match else "N/A"
            return [product_name, f"USD {price}", rating]
    except Exception as e:
        print(f"Error: {e}")
        return [product_name, "Error", "Error"]

def scrape_skinlovecream_product(url, product_name):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        price_tag = soup.find("meta", property="og:price:amount")
        currency_tag = soup.find("meta", property="og:price:currency")
        rating_div = soup.find("div", class_="jdgm-rev-widg")

        price = price_tag["content"] if price_tag else "N/A"
        currency = currency_tag["content"] if currency_tag else "N/A"
        rating = rating_div["data-average-rating"] if rating_div else "N/A"

        return [product_name, f"{currency} {price}", rating]
    else:
        return [product_name, "N/A", "N/A"]

def scrape_walgreens_product(url, product_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            html_content = str(soup)

            price_pattern = r'\$\d+\.\d{2}'
            price = re.findall(price_pattern, html_content)
            found_price = price[0] if price else "Price not found"

            rating_pattern = r'"shortMessage":"([\d.]+ out of 5)"'
            rating_match = re.search(rating_pattern, html_content)
            found_rating = rating_match.group(1).replace(" out of 5", "") if rating_match else "Rating not found"

            return [product_name, found_price, found_rating]
        else:
            return [product_name, "Error", "Error"]
    except Exception as e:
        return [product_name, "Error", "Error"]

def scrape_ultabeauty_product(url, product_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text

            price_pattern = r'<span class="Text-ds Text-ds--title-6 Text-ds--left Text-ds--black">\$(\d+\.\d{2}) </span>'
            rating_pattern = r'"aggregateRating":{"@type":"AggregateRating","ratingValue":(\d+\.\d)'

            price_match = re.search(price_pattern, html_content)
            rating_match = re.search(rating_pattern, html_content)

            found_price = f'${price_match.group(1)}' if price_match else "Price not found"
            found_rating = rating_match.group(1) if rating_match else "Rating not found"

            return [product_name, found_price, found_rating]
        else:
            return [product_name, "Error", "Error"]
    except Exception as e:
        return [product_name, "Error", "Error"]

# Combined scraping function
def combined_scrape():
    print(f" Scraping started for Kohls, Lovelyskin, Skinlovecream, Walgreens, Ultabeauty at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Kohls products
    kohls_products = [
        ("https://www.kohls.com/product/prd-5050008/olaplex-no-7-bonding-hair-oil.jsp?skuid=17911732&CID=shopping15&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=17911732&utm_campaignid=20485373179&CID=shopping30&utm_campaign=MediumSSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20485373179&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTchpQ_E0QOv8B4c1oPWYYi4Fr-60QRdAuCuDpeJorwSAXAQ-4posn0aAn9sEALw_wcB&gclsrc=aw.ds", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
       ("https://www.kohls.com/product/prd-5050005/olaplex-no-4-bond-maintenance-shampoo.jsp?skuid=67270281&CID=shopping30&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=67270281&utm_campaignid=20507263090&CID=shopping30&utm_campaign=SSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20507263090&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTdfObfjg7NJgEgbftJ7zSZEO5Kbm42pdU7mpo-oP61ddx_saVyqJhIaAu0zEALw_wcB&gclsrc=aw.ds", "Olaplex No. 4 Bond Maintenance Shampoo 8.5 fl oz"),
       ("https://www.kohls.com/product/prd-5050006/olaplex-no-5-bond-maintenance-conditioner.jsp?skuid=65150746&CID=shopping15&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=65150746&utm_campaignid=20485373179&CID=shopping30&utm_campaign=MediumSSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20485373179&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTfHKiLgn1tIc_JhVFD_SSPI-Xs3KzDqdR6VxoCl5Q-Yyc9RkmDE6cMaAv-7EALw_wcB&gclsrc=aw.ds", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
       ("https://www.kohls.com/product/prd-5532929/olaplex-no-6-bond-smoother-reparative-styling-creme.jsp?skuid=71250025&CID=shopping30&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=71250025&utm_campaignid=20507263090&CID=shopping30&utm_campaign=SSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20507263090&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTf94yPAHlFCvJ--MUTq11aFNtqulBRmfO8iTpUh6lkngel6vXBjo1YaAkedEALw_wcB&gclsrc=aw.ds", "Olaplex No. 6 Bond Smoother 3.3 fl oz"),
       ("https://www.kohls.com/product/prd-5050004/olaplex-no-3-hair-repair-perfector.jsp?skuid=49701743&CID=shopping30&utm_campaign=HAIR&utm_medium=CSE&utm_source=google&utm_product=49701743&utm_campaignid=20507263090&CID=shopping30&utm_campaign=SSCSephora&utm_medium=CSE&utm_source=google&utm_campaignid=20507263090&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTcR-zMBFZhx7NXZdAtSKIU37Xgyp8pH06g4AU2bwteC4KqcbF-j7ToaAg_dEALw_wcB&gclsrc=aw.ds", "No. 3 Hair Perfector Treatment 3.3 fl oz")

    ]
    kohls_data = [scrape_kohls_product(url, name) for url, name in kohls_products]
    print(tabulate(kohls_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    # Lovelyskin products
    lovelyskin_products = [
        ("https://www.lovelyskin.com/o/olaplex-no-7-bonding-oil", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
        ("https://www.lovelyskin.com/o/olaplex-no-4-bond-maintenance-shampoo?ref=gbase", "Olaplex No. 4 Bond Maintenance Shampoo Size: 8.5 fl oz"),
        ("https://www.lovelyskin.com/o/olaplex-no-5-bond-maintenance-conditioner", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
        ("https://www.lovelyskin.com/o/olaplex-no-6-bond-smoother?ref=gbase&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTeAH7knJVVWPFjytwdkhdksfBX_0uNRx8OhQJyiK-wylUZE8x6TRlAaAmIMEALw_wcB", "Olaplex No. 6 Bond Smoother Size: 3.3 fl oz"),
        ("https://www.lovelyskin.com/o/olaplex-no-3-hair-perfector", "No. 3 Hair Perfector Treatment Size: 3.3 fl oz")

    ]
    lovelyskin_data = [scrape_lovelyskin_product(url, name) for url, name in lovelyskin_products]
    print(tabulate(lovelyskin_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    # Skinlovecream products
    skinlovecream_products = [
        ("https://skinlovecream.com/products/olaplex-no-7-bonding-oil?variant=42577740792054&currency=USD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTflJA-jxmndNNH2Vb43Shu-4vB_OE95yZOrzWTaHrioodCLZA3iiOcaAmuQEALw_wcB", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
        ("https://skinlovecream.com/products/olaplex-no-4-bond-maintenance-shampoo?variant=42577567842550&currency=USD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTddLO8wcbqkX1B87HEzC3R9w1cLS6ZcrV_ZmemfjgxqMz3XM3QS1pkaAuM8EALw_wcB", "Olaplex No. 4 Bond Maintenance Shampoo Size: 8.5 fl oz"),
        ("https://skinlovecream.com/products/olaplex-no-5-bond-maintenance-conditioner?variant=42577632395510", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
        ("https://skinlovecream.com/products/olaplex-no-6-bond-smoother?variant=42577691246838", "Olaplex No. 6 Bond Smoother Size: 3.3 fl oz"),
        ("https://skinlovecream.com/products/olaplex-no-3-hair-perfector?variant=42577661886710", "No. 3 Hair Perfector Treatment Size: 3.3 fl oz")

    ]
    skinlovecream_data = [scrape_skinlovecream_product(url, name) for url, name in skinlovecream_products]
    print(tabulate(skinlovecream_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    # Walgreens products
    walgreens_products = [
        ("https://www.walgreens.com/store/c/olaplex-no.-7-bonding-hair-oil/ID=300425433-product", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
        ("https://www.walgreens.com/store/c/olaplex-no.-4-bond-maintenance-shampoo/ID=300427190-product?ext=gooFY23_GOO_RET_RETAILDEMANDGEN_Performance%2BMax%2B-%2BBeauty_REV_SRC_PMAX_PMAX_NA_PMAX_ENG__pla_online&gclsrc=aw.ds&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTdLmST044rktqFajOY_dfaV89unsRjC6SaNBqgB-vJcmQ2dkMOCm8saAj0PEALw_wcB", "Olaplex No. 4 Bond Maintenance Shampoo Size: 8.5 fl oz"),
        ("https://www.walgreens.com/store/c/olaplex-no.-5-bond-maintenance-conditioner/ID=300429026-product", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
        ("https://www.walgreens.com/store/c/olaplex-no.-6-bond-smoother/ID=300427188-product?ext=gooFY%7E24_LB%7ERDG_CH%7ESEARCH_CN%7ERDG_Catch%2BAll%2B-%2BRSC_CA%7EFOS_MT%7EPLA_LG%7EEN1_RE%7ENA_MK%7EGM_OB%7ESALES_PK%7EOMNIREV_KT%7ENA_KM%7ENA_AS%7EGOOFY%7E24_%2BAS%7EGOO_FF%7ECatch%2BAll%2B-%2BAll%2BProducts_DT%7ENA_LG%7EEN1_MK%7EGM_SA%7ETP_SI%7ENA_TC%7EKEY_PB%7EGO_PK%7EOMNIREV__pla_online&gclsrc=aw.ds&gad_source=1&gclid=Cj0KCQiAmNeqBhD4ARIsADsYfTf1aX6NoZO1MRQti8xYoLNfTJyn1TvXY7yyxLG5UzGvEWPOMbuSvN0aAhSzEALw_wcB", "Olaplex No. 6 Bond Smoother Size: 3.3 fl oz"),
        ("https://www.walgreens.com/store/c/olaplex-no.-3-hair-perfector-treatment/ID=prod6310485-product", "No. 3 Hair Perfector Treatment Size: 3.3 fl oz")

    ]
    walgreens_data = [scrape_walgreens_product(url, name) for url, name in walgreens_products]
    print(tabulate(walgreens_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    # Ultabeauty products
    ultabeauty_products = [
        ("https://www.ulta.com/p/no7-bonding-oil-pimprod2029354?sku=2591018", "Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz"),
        ("https://www.ulta.com/p/no4-bond-maintenance-shampoo-pimprod2029348?sku=2591015", "Olaplex No. 4 Bond Maintenance Shampoo Size: 8.5 fl oz"),
        ("https://www.ulta.com/p/no5-bond-maintenance-conditioner-pimprod2029349?sku=2591016", "Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz"),
        ("https://www.ulta.com/p/no6-bond-smoother-pimprod2030408?sku=2592990", "Olaplex No. 6 Bond Smoother Size: 3.3 fl oz"),
        ("https://www.ulta.com/p/no3-hair-perfector-pimprod2029343?sku=2591014", "No. 3 Hair Perfector Treatment Size: 3.3 fl oz")

    ]
    ultabeauty_data = [scrape_ultabeauty_product(url, name) for url, name in ultabeauty_products]
    print(tabulate(ultabeauty_data, headers=["Product Name", "Price", "Rating"], tablefmt="fancy_grid"))

    print(f" Scraping ended at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# Schedule the combined scraping task
schedule.every().day.at("02:03:00").do(combined_scrape)

# Run the scheduler in a loop
while True:
    schedule.run_pending()
    time.sleep(1)

 Scraping started for Kohls, Lovelyskin, Skinlovecream, Walgreens, Ultabeauty at 2024-03-29 02:03:00

╒════════════════════════════════════════════════════════════╤═════════╤══════════╕
│ Product Name                                               │ Price   │   Rating │
╞════════════════════════════════════════════════════════════╪═════════╪══════════╡
│ Olaplex No. 7 Bonding Hair Oil Size: 1.0 oz                │ $30.00  │      4.3 │
├────────────────────────────────────────────────────────────┼─────────┼──────────┤
│ Olaplex No. 4 Bond Maintenance Shampoo 8.5 fl oz           │ $30.00  │      4.2 │
├────────────────────────────────────────────────────────────┼─────────┼──────────┤
│ Olaplex No. 5 Bond Maintenance Conditioner Size: 8.5 fl oz │ $30.00  │      4.1 │
├────────────────────────────────────────────────────────────┼─────────┼──────────┤
│ Olaplex No. 6 Bond Smoother 3.3 fl oz                      │ $30.00  │      4.4 │
├─────────────────────────────────────────────────────────

KeyboardInterrupt: 